In [ ]:
# 불러올 라이브러리

import requests, time, csv, openpyxl
import datetime as dt
from bs4 import BeautifulSoup
from selenium import webdriver

## 구미도서관 노트북 열람실 crawling
>### ver1
1. 3층 노트북 열람실만 긁어옴
1. loop 횟수의 지정이 필요함
1. 향후. 모든 열람실 긁어오기, 남은 좌석과 밀접한 연관이 있는 변수 집어넣기

In [ ]:

# 반복 횟수(loop) : 1이 1분이라고 보면 됨
loop = 1


#-----------크롬드라이버 실행 및 xlsx 파일 생성 후, 칼럼 부여-------

chrome_driver = '/Users/1000/Downloads/chromedriver'
driver = webdriver.Chrome(chrome_driver)

wb = openpyxl.Workbook()
sheet1 = wb['Sheet']
sheet1.title = 'data'
sheet1.append(["status", "tot_s","using_s","left_s","wait_s",
               "month","day","weekday","workingday","apm","hour","timestamp"])

# 페이지 불러오기, 서버상태 체크, html 파싱, 칼럼에 필요한 tag(data) 수집
# 날짜와 시간은 별도의 작업을 통해서 int 변환 
# week = [0월, 1화, 2수, 3목, 4금, 5토, 6일] 
# workingday : 0=평일, 1=주말
# apm : 0=오전, 1=오후
for t in range(1,loop+1,1):
    lib_url = 'http://211.253.111.235:8068/SNLib/Seat_Info/3_count.asp'
    driver.get(lib_url)
    resp = requests.get(lib_url)
    
    if resp.status_code == 200  :
        time.sleep(1)
        src = driver.page_source
        soup = BeautifulSoup(src,'html.parser')
        
        try :
            status = soup.find_all('td')[0].get_text()
            tot_s = int(soup.find_all('td')[1].get_text())
            using_s= int(soup.find_all('td')[2].get_text())
            left_s= int(soup.find_all('td')[3].get_text())
            wait_s= int(soup.find_all('td')[4].get_text())
            timestamp = soup.find_all('td')[6].get_text()

            year, month, day = timestamp.split()[0].split('-')
            week = dt.datetime.strptime(timestamp.split()[0],
                                                 "%Y-%m-%d").date().weekday()
            workingday = 0
            if week >= 5 :
                workingday = 1

            def apm_(x):
                if x.split()[1] == "오전" : return 0
                else : return 1
            apm = apm_(timestamp)

            # 서버시간이 13시 대신 오후1시로 표시되서 apm값과 함께 hour_2 산출
            hour_1 = int(timestamp.split()[2].split(':')[0])
            hour_2 = hour_1 + (lambda x : 12 if x == 1 else 0)(apm)

            sheet1.append([status, tot_s, using_s, left_s, wait_s,
                           int(month),int(day),week,workingday,apm,hour_2,timestamp])
            # 리스트 자체를 append에 추가함.
            # openpyxl에서 append는 마지막행(데이터가 있는) 다음에 데이터를 추가해줌!!굿

            time.sleep(60)

            if t/loop*100 in range(10,110,10):
                print('현재 {}% 완료'.format(t/loop*100))
            
        except:
            print("서버 튕김으로 쉬는 중...60초후 재개합니다.")
            time.sleep(60)
            src = driver.page_source
            soup = BeautifulSoup(src,'html.parser')
            
            status = soup.find_all('td')[0].get_text()
            tot_s = int(soup.find_all('td')[1].get_text())
            using_s= int(soup.find_all('td')[2].get_text())
            left_s= int(soup.find_all('td')[3].get_text())
            wait_s= int(soup.find_all('td')[4].get_text())
            timestamp = soup.find_all('td')[6].get_text()

            year, month, day = timestamp.split()[0].split('-')
            week = dt.datetime.strptime(timestamp.split()[0],
                                                 "%Y-%m-%d").date().weekday()
            workingday = 0
            if week >= 5 :
                workingday = 1

            def apm_(x):
                if x.split()[1] == "오전" : return 0
                else : return 1
            apm = apm_(timestamp)

            # 서버시간이 13시 대신 오후1시로 표시되서 apm값과 함께 hour_2 산출
            hour_1 = int(timestamp.split()[2].split(':')[0])
            hour_2 = hour_1 + (lambda x : 12 if x == 1 else 0)(apm)

            sheet1.append([status, tot_s, using_s, left_s, wait_s,
                           int(month),int(day),week,workingday,apm,hour_2,timestamp])
            # 리스트 자체를 append에 추가함.
            # openpyxl에서 append는 마지막행(데이터가 있는) 다음에 데이터를 추가해줌!!굿

            time.sleep(60)

            if t/loop*100 in range(10,110,10):
                print('현재 {}% 완료'.format(t/loop*100))

    else :
        print("서버 오류로 잠시 쉬는 중...60초후 재개합니다.")
        time.sleep(60)
        continue
    
driver.close()
wb.save('lib_status_test.csv')